<a href="https://colab.research.google.com/github/aswit3/Analystics_Vidhya_Sentiment_Analysis_Medical/blob/master/Twitter_Sentiment_Analysis_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Twitter-Sentiment-Analysis**

https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import re # for regular expressions
import pandas as pd 
pd.set_option("display.max_colwidth", 200)
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk # for text manipulation
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [0]:
train  = pd.read_csv("drive/My Drive/datasets/Twitter-Sentiment-Analysis/train_E6oV3lV.csv") #/content/drive/My Drive/datasets/Twitter-Sentiment-Analysis/test_tweets_anuFYb8.csv
test = pd.read_csv("drive/My Drive/datasets/Twitter-Sentiment-Analysis/test_tweets_anuFYb8.csv")

In [0]:
train.shape, test.shape

((31962, 3), (17197, 2))

In [0]:
train["label"].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [0]:
combi = train.append(test, ignore_index=True)
combi.shape

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(49159, 3)

In [0]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [0]:
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*") 
#combi.head()

In [0]:
combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
#combi.head(10)

In [0]:
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [0]:
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split()) # tokenizing

In [0]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

In [0]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
    
combi['tidy_tweet'] = tokenized_tweet

In [0]:
combi['tidy_tweet'].shape

(49159,)

In [0]:
# function to collect hashtags
def hashtag_extract(x):
    hashtags = []
    # Loop over the words in the tweet
    for i in x:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)

    return hashtags

In [0]:
# extracting hashtags from non racist/sexist tweets

HT_regular = hashtag_extract(combi['tidy_tweet'][combi['label'] == 0])

# extracting hashtags from racist/sexist tweets
HT_negative = hashtag_extract(combi['tidy_tweet'][combi['label'] == 1])

# unnesting list
HT_regular = sum(HT_regular,[])
HT_negative = sum(HT_negative,[])

print(len(HT_regular), len(HT_negative))

68425 4616


In [0]:
import gensim

In [0]:
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split()) # tokenizing

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=200, # desired no. of features/independent variables 
            window=5, # context window size
            min_count=2,
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            seed = 34)

model_w2v.train(tokenized_tweet, total_examples= len(combi['tidy_tweet']), epochs=20)

(6510028, 7536020)

In [0]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    
    if count != 0:
        vec /= count
    
    return vec


In [0]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200))

for i in range(len(tokenized_tweet)):
    #print(tokenized_tweet[i])
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)

wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(49159, 200)

In [0]:
from sklearn.model_selection import train_test_split


train_w2v = wordvec_df.iloc[:31962,:]
test_w2v = wordvec_df.iloc[31962:,:]

_, _, ytrain, yvalid = train_test_split(train_w2v, train['label'],  
                                                          random_state=42, 
                                                          test_size=0.3)



print(train_w2v.shape, train['label'].shape)

X_train = train_w2v.to_numpy()
Y_train = train['label'].to_numpy()

(31962, 200) (31962,)


In [0]:
import numpy as np
import tensorflow as tf
n_inputs = 200
n_neurons = 512
n_timesteps = 2

learning_rate = 0.001
num_steps = 500

n_hidden_1 = n_neurons
n_hidden_2 = n_neurons
num_classes = 2

In [0]:
no_of_samples = X_train.shape[0]

EPOCHS = 10
BATCH_SIZE = 2

N_BATCHES = no_of_samples//BATCH_SIZE
X_train = X_train.reshape(N_BATCHES, BATCH_SIZE, 200)
Y_train = Y_train.reshape(N_BATCHES, BATCH_SIZE, 1)

In [0]:
X_train.shape, Y_train.shape

((15981, 2, 200), (15981, 2, 1))

In [0]:
X = tf.placeholder(tf.float32, [None, BATCH_SIZE, n_inputs])
Y = tf.placeholder(tf.int32, [None, BATCH_SIZE, 1])

def batch(iterable, input_batch, target_batch, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield input_batch[ndx:min(ndx + n, l)], target_batch[ndx:min(ndx + n, l)]
        
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_inputs, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
outputs = tf.matmul(layer_2, weights['out']) + biases['out']

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
prediction = tf.nn.softmax(outputs)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(EPOCHS):
  sess.run(outputs, feed_dict={X: X_train, Y:Y_train})
  
  for x, y in batch(range(0, N_BATCHES), X_train, Y_train, 3):
    loss, acc = sess.run([loss_op, accuracy], feed_dict={X: x, Y:y})
  print(loss, acc)

139.81657 0.8333333
139.81657 0.8333333
139.81657 0.8333333
139.81657 0.8333333
139.81657 0.8333333
139.81657 0.8333333
139.81657 0.8333333
139.81657 0.8333333
139.81657 0.8333333
139.81657 0.8333333


In [0]:
test_w2v = wordvec_df.iloc[31963:,:]
test_w2v = test_w2v.to_numpy()
test_w2v = test_w2v.reshape(test_w2v.shape[0]//BATCH_SIZE, BATCH_SIZE, 200)
prediction = sess.run([prediction], feed_dict={X: test_w2v})

#print(prediction, prediction.shape)
# test['label'] = test_pred
# submission = test[['id','label']]
# submission.to_csv('model_result.csv', index=False)


In [0]:
type(prediction)

list

In [0]:
prediction = np.array(prediction)
prediction.shape, prediction

((1, 8598, 2, 2), array([[[[1.0000000e+00, 1.0774032e-09],
          [1.0000000e+00, 0.0000000e+00]],
 
         [[0.0000000e+00, 1.0000000e+00],
          [0.0000000e+00, 1.0000000e+00]],
 
         [[0.0000000e+00, 1.0000000e+00],
          [1.0000000e+00, 0.0000000e+00]],
 
         ...,
 
         [[0.0000000e+00, 1.0000000e+00],
          [0.0000000e+00, 1.0000000e+00]],
 
         [[1.0000000e+00, 0.0000000e+00],
          [1.0000000e+00, 0.0000000e+00]],
 
         [[0.0000000e+00, 1.0000000e+00],
          [0.0000000e+00, 1.0000000e+00]]]], dtype=float32))

In [0]:
prediction = prediction.reshape(17196,2)

In [0]:
prediction.shape, prediction

((17196, 2), array([[1.0000000e+00, 1.0774032e-09],
        [1.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 1.0000000e+00],
        ...,
        [1.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 1.0000000e+00],
        [0.0000000e+00, 1.0000000e+00]], dtype=float32))

In [0]:
prediction = prediction.argmax(axis = -1)

In [0]:
prediction.shape

()